In [5]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import csv
import pandas as pd

In [6]:
School_Districts = [
    #Blue Springs R-IV
    "https://ocrdata.ed.gov/Page?t=d&eid=30674&syk=6&pid=736",
    #Blue Valley Unified School District 229
    "https://ocrdata.ed.gov/Page?t=d&eid=29265&syk=6&pid=736",
    #Bonner Springs Unified School District 204
    "https://ocrdata.ed.gov/Page?t=d&eid=31802&syk=6&pid=736",
    #CENTER 58
    "https://ocrdata.ed.gov/Page?t=d&eid=48284&syk=6&pid=736",
    #CLINTON CO. R-III
    "https://ocrdata.ed.gov/Page?t=d&eid=27990&syk=6&pid=736",
    #DESOTO 73
    "https://ocrdata.ed.gov/Page?t=d&eid=31816&syk=6&pid=736",
    #EXCELSIOR SPRINGS 40
    "https://ocrdata.ed.gov/Page?t=d&eid=30709&syk=6&pid=736",
    #FORT OSAGE R-I
    "https://ocrdata.ed.gov/Page?t=d&eid=27902&syk=6&pid=736",
    #Gardner Edgerton Unified School District 231
    "https://ocrdata.ed.gov/Page?t=d&eid=31828&syk=6&pid=736",
    #GRAIN VALLEY R-V
    "https://ocrdata.ed.gov/Page?t=d&eid=27909&syk=6&pid=736",
    #GRANDVIEW C-4
    "https://ocrdata.ed.gov/Page?t=d&eid=27910&syk=6&pid=736",
    #HICKMAN MILLS C-1
    "https://ocrdata.ed.gov/Page?t=d&eid=27919&syk=6&pid=736",
    #INDEPENDENCE 30
    "https://ocrdata.ed.gov/Page?t=d&eid=27926&syk=6&pid=736",
    #KANSAS CITY 33
    "https://ocrdata.ed.gov/Page?t=d&eid=27932&syk=6&pid=736",
    #Kansas City Unified School District 500
    "https://ocrdata.ed.gov/Page?t=d&eid=29221&syk=6&pid=736",
    #KEARNEY R-I
    "https://ocrdata.ed.gov/Page?t=d&eid=27933&syk=6&pid=736",
    #LAWSON R-XIV
    "https://ocrdata.ed.gov/Page?t=d&eid=27942&syk=6&pid=736",
    #LEE'S SUMMIT R-VII
    "https://ocrdata.ed.gov/Page?t=d&eid=27944&syk=6&pid=736",
    #Liberty
    "https://ocrdata.ed.gov/Page?t=d&eid=27946&syk=6&pid=736",
    #LONE JACK C-6
    "https://ocrdata.ed.gov/Page?t=d&eid=48395&syk=6&pid=736",
    #NORTH KANSAS CITY 74
    "https://ocrdata.ed.gov/Page?t=d&eid=27972&syk=6&pid=736",
    #NORTH PLATTE CO. R-I
    "https://ocrdata.ed.gov/Page?t=d&eid=48441&syk=6&pid=736",
    #OAK GROVE R-VI
    "https://ocrdata.ed.gov/Page?t=d&eid=27975&syk=6&pid=736",
    #Olathe
    "https://ocrdata.ed.gov/Page?t=d&eid=29245&syk=6&pid=736",
    #PARK HILL
    "https://ocrdata.ed.gov/Page?t=d&eid=27983&syk=6&pid=736",
    #Piper !!No Piper School District for 2011
    #"https://ocrdata.ed.gov/Page?t=d&eid=47007&syk=6&pid=736",
    #PLATTE CO. R-III
    "https://ocrdata.ed.gov/Page?t=d&eid=27989&syk=6&pid=736",
    #RAYTOWN C-2
    "https://ocrdata.ed.gov/Page?t=d&eid=27995&syk=6&pid=736",
    #Shawnee Mission Public Schools Unified School District
    "https://ocrdata.ed.gov/Page?t=d&eid=29262&syk=6&pid=736",
    #SMITHVILLE R-II
    "https://ocrdata.ed.gov/Page?t=d&eid=48496&syk=6&pid=736",
    #Spring Hill Unified School District 230
    "https://ocrdata.ed.gov/Page?t=d&eid=47027&syk=6&pid=736",
    #Turner Unified School District 202
    "https://ocrdata.ed.gov/Page?t=d&eid=29270&syk=6&pid=736",
    #WEST PLATTE CO. R-II
    "https://ocrdata.ed.gov/Page?t=d&eid=48539&syk=6&pid=736"
]

In [7]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [8]:
#import decimal
data = []

for district in School_Districts:

    #Open up the page and wait for JS to load content
    driver = webdriver.Firefox()
    driver.get(district)
    time.sleep(5)
    htmlSource = driver.page_source
    soup = BeautifulSoup(htmlSource, "html.parser")
    
    District_Name = soup.find("span", { "class" : "spanSchool" }).text
    
    OSS = soup.find("td", { "id" : "tdOutOfSchoolSuspensions" })
    OSS_total = OSS.find("text", { "class" : "highcharts-subtitle"}).text
    OSS_total_number = int(re.sub("[^0-9]", "", OSS_total))
    
    OSS_legend = OSS.find("g", { "class" : "highcharts-legend"})
    cells = OSS_legend.findAll("g", { "class" : "highcharts-legend-item"})   
    
    percent_Black_OSS = float(re.findall("\d+\.\d+", cells[2].find(text=True).rstrip())[0])/100
    number_Black_OSS = int(round(percent_Black_OSS * OSS_total_number))
    
    
    percent_White_OSS = float(re.findall("\d+\.\d+", cells[6].find(text=True).rstrip())[0])/100
    number_White_OSS = int(round(percent_White_OSS * OSS_total_number))
    
    enroll = soup.findAll("td", { "class" : "number" })
    
    #Enrollment
    Total_Enrolled = enroll[1].find(text=True).rstrip()
    Total_Enrolled_number = int(re.sub("[^0-9]", "", Total_Enrolled))
    
    percent = soup.findAll("td", { "class" : "percent" })
    
    #Percent Black Enrollment
    percent_Black = round(float(re.findall("\d+\.\d+", percent[2].find(text=True).rstrip())[0])/100,3)
    number_Black = int(round(percent_Black * Total_Enrolled_number))
    
    
    #Percent White Enrollment
    percent_White = round(float(re.findall("\d+\.\d+", percent[6].find(text=True).rstrip())[0])/100,3)
    number_White = int(round(percent_White * Total_Enrolled_number))
    
    #Suspension_Rates
    BlackSusRate = round((number_Black_OSS / number_Black),3)
    WhiteSusRate = round((number_White_OSS / number_White),3)
    
    #Suspension Gap
    Suspension_Gap = (BlackSusRate - WhiteSusRate)*100
        
    data.append((District_Name,
                 OSS_total_number,
                 percent_White_OSS,
                 number_White_OSS,
                 percent_Black_OSS,
                 number_Black_OSS,
                 Total_Enrolled_number,
                 percent_Black,
                 number_Black,
                 percent_White,
                 number_White,
                 BlackSusRate,
                 WhiteSusRate,
                 Suspension_Gap
                ))
    
    driver.quit()

In [9]:
with open('CRDC_Suspennsions_2011.csv', 'a') as csv_file:
    writer = csv.writer(csv_file)
 # The for loop
    writer.writerow([
        "District_Name",
        "total_suspensions", 
        "%White_OSS",
        "num_White_OSS",
        "%Black_OSS",
        "num_Black_OSS",
        "Enrollment",
        "%Black",
        "num_Black",
        "%White",
        "num_White",
        "Black_Sus_Rate",
        "White_Sus_Rate",
        "Suspension_Gap"
    ])
    for District_Name, OSS_total_number, percent_White_OSS, number_White_OSS, percent_Black_OSS, number_Black_OSS, Total_Enrolled_number, percent_Black, number_Black, percent_White, number_White, BlackSusRate, WhiteSusRate, Suspension_Gap   in data:
        writer.writerow([
                 District_Name,
                 OSS_total_number,
                 percent_White_OSS,
                 number_White_OSS,
                 percent_Black_OSS,
                 number_Black_OSS,
                 Total_Enrolled_number,
                 percent_Black,
                 number_Black,
                 percent_White,
                 number_White,
                 BlackSusRate,
                 WhiteSusRate,
                 Suspension_Gap
        ])